In [21]:
from audiomentations import ApplyImpulseResponse, AddGaussianNoise, TimeStretch, PitchShift, Compose
import itertools
import numpy as np

In [4]:
augmentations = {
    "timestretch": TimeStretch(min_rate=0.5, max_rate=2.0, p=1.0, leave_length_unchanged=False),
    "pitchshift": PitchShift(min_semitones=-8, max_semitones=8, p=1.0),
    "gaussian": AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1.0),
    "rir": ApplyImpulseResponse(ir_path="/home/joregan/.psst/rir", p=1.0),
}

/home/joregan/miniconda3/envs/fs2/lib/python3.8/site-packages/audiomentations/augmentations/apply_impulse_response.py:48: UserWarning: The default value of leave_length_unchanged will change from False to True in a future version of audiomentations. You can set the value explicitly to remove this warning for now.
  warnings.warn(


In [5]:
all2=list(itertools.combinations_with_replacement(augmentations.keys(), 2))
all3=list(itertools.combinations_with_replacement(augmentations.keys(), 3))

In [6]:
all2 = [a for a in all2 if a[0] != a[1]]

In [7]:
all3 = [a for a in all3 if(a[0] != a[1] and a[1] != a[2] and a[0] != a[2])]

In [27]:
combinations = {}
for item in all2:
    key = "-".join(item)
    val = Compose([augmentations[item[0]], augmentations[item[1]]])
    combinations[key] = val
for item in all3:
    key = "-".join(item)
    val = Compose([augmentations[item[0]], augmentations[item[1]], augmentations[item[2]]])
    combinations[key] = val
combinations["-".join(augmentations.keys())] = Compose([v for v in augmentations.values()])

In [28]:
combinations

{'timestretch-pitchshift': <audiomentations.core.composition.Compose at 0x7fc658e47100>,
 'timestretch-gaussian': <audiomentations.core.composition.Compose at 0x7fc658e47f10>,
 'timestretch-rir': <audiomentations.core.composition.Compose at 0x7fc658e47fa0>,
 'pitchshift-gaussian': <audiomentations.core.composition.Compose at 0x7fc658e27430>,
 'pitchshift-rir': <audiomentations.core.composition.Compose at 0x7fc658e27880>,
 'gaussian-rir': <audiomentations.core.composition.Compose at 0x7fc658e272b0>,
 'timestretch-pitchshift-gaussian': <audiomentations.core.composition.Compose at 0x7fc658e47970>,
 'timestretch-pitchshift-rir': <audiomentations.core.composition.Compose at 0x7fc658e275b0>,
 'timestretch-gaussian-rir': <audiomentations.core.composition.Compose at 0x7fc658e27340>,
 'pitchshift-gaussian-rir': <audiomentations.core.composition.Compose at 0x7fc658e27160>,
 'timestretch-pitchshift-gaussian-rir': <audiomentations.core.composition.Compose at 0x7fc65bee4190>}

In [29]:
_DIR = "/home/joregan/psst-data/psst-data-2022-03-02/train"
_TSV = "/home/joregan/.psst/out/data/psst-fairseq/train.tsv"

In [30]:
TIMITFILES = []
with open(_TSV) as f:
    for line in f.readlines():
        if line.startswith("TEST/") or line.startswith("TRAIN/"):
            parts = line.split("\t")
            TIMITFILES.append(parts[0])

In [31]:
from pathlib import Path
_DIR_PATH = Path(_DIR)

In [32]:
import soundfile as sf

In [33]:
check = [a.replace("TRAIN/", "").replace("TEST/", "") for a in TIMITFILES]
assert len(check) == len(set(check))

In [34]:
all = {**augmentations, **combinations}

In [38]:
_TSV_DIR = Path("/home/joregan/.psst/out/data/psst-fairseq/")

In [40]:
COMMON_PATH = "/home/joregan/psst-data/psst-data-2022-03-02/train/"

In [ ]:
for file in TIMITFILES:
    if not (file.startswith("TRAIN/") or file.startswith("TEST/")):
        continue
    filepath = _DIR_PATH / file
    filestr = str(filepath)
    raw_audio, sr = sf.read(filestr)
    orig_audio = np.array(raw_audio)
    for aug in all.keys():
        out_tsv = _TSV_DIR / f"{str(aug)}.tsv"
        if "/TRAIN/" in filestr:
            outfile = filestr.replace("/TRAIN/", f"/{aug}/")
        elif "/TEST/" in filestr:
            outfile = filestr.replace("/TEST/", f"/{aug}/")
        else:
            print(f"Error reading file: {filestr}")
            continue
        outpath = Path(outfile)
        outpath.parent.mkdir(parents=True, exist_ok=True)
        augmenter = all[aug]
        try:
            newaudio = augmenter(samples=orig_audio, sample_rate=sr)
        except AttributeError:
            raise AttributeError("Error with augmentation", aug)
        out_frames = len(newaudio)
        writable_filestr = outfile.replace(COMMON_PATH, "")
        with open(out_tsv, "a") as tsvf:
            tsvf.write(f"{writable_filestr}\t{out_frames}\n")
        sf.write(outfile, newaudio, sr)


In [41]:
all_the_things = """
gaussian-rir
gaussian
pitchshift-gaussian-rir
pitchshift-gaussian
pitchshift-rir
pitchshift
rir
timestretch-gaussian-rir
timestretch-gaussian
timestretch-pitchshift-gaussian-rir
timestretch-pitchshift-gaussian
timestretch-pitchshift-rir
timestretch-pitchshift
timestretch-rir
timestretch
"""